In [1]:
from pathlib import Path
import sqlite3
from collections import namedtuple

import numpy as np
import sklearn
import pandas as pd
from pandas import Int64Index

In [2]:
path_data = Path('sql')
path_data.mkdir(exist_ok=True)
db_file = path_data / 'db.sqlite'
with sqlite3.connect(str(db_file)) as con:
    df = pd.read_sql(con=con, sql='select s.article_id_1, s.article_id_2, s.permid, s.sklearn, a1.source_id as source_id_1, a2.source_id as source_id_2 from similarities s inner join article a1 on s.article_id_1 = a1.article_id inner join article a2 on s.article_id_2 = a2.article_id')
df.head(5)

,article_id_1,article_id_2,permid,sklearn,source_id_1,source_id_2
0,2,1,0,0.247143,1,1
1,3,1,0,0.434966,1,1
2,3,2,0,0.276686,1,1
3,4,1,0,0.491659,1,1
4,4,2,0,0.359131,1,1


In [3]:
threshold = 0.2
df = df[df.sklearn > threshold]
df = df.sort_values('sklearn', ascending=False)
df = df[df.source_id_1 != df.source_id_2]
df = df.reset_index(drop=True)
# df.article_id_1 = df.article_id_1.astype(int)
# df.article_id_2 = df.article_id_2.astype(int)
# df.source_id_1 = df.source_id_1.astype(int)
# df.source_id_2 = df.source_id_2.astype(int)
df.head(5)

,article_id_1,article_id_2,permid,sklearn,source_id_1,source_id_2
0,36,13,0,0.737309,4,2
1,13,1,0,0.726409,2,1
2,31,11,0,0.701009,4,2
3,37,19,0,0.700437,4,2
4,79,77,0,0.643098,1,4


In [4]:
cliques = []
while len(cliques) < 2 and len(df) >= 4:
    c = [None, None, None, None]

    first_row = df.iloc[0]
    df = df.drop(df.head(1).index)

    c[int(first_row.source_id_1) - 1] = int(first_row.article_id_1)
    c[int(first_row.source_id_2) - 1] = int(first_row.article_id_2)
    c
    index = 0

    while c[0] is None or c[1] is None or c[2] is None or c[3] is None and len(df) > index:
        df = df.reset_index(drop=True)
#         print(index)
        new_row = df.iloc[index]
        if c[int(new_row.source_id_1) - 1] is None and c[int(new_row.source_id_2) - 1] is not None:
            c[int(new_row.source_id_1) - 1] = int(new_row.article_id_1)
            df = df.drop(Int64Index([index+1], dtype='int64'))
        elif c[int(new_row.source_id_1) - 1] is not None and c[int(new_row.source_id_2) - 1] is None:
            c[int(new_row.source_id_2) - 1] = int(new_row.article_id_2)
            df = df.drop(Int64Index([index+1], dtype='int64'))
        else:
            index += 1
    for i in range(4):
        df = df[(df.article_id_1 != c[i]) & (df.article_id_2 != c[i])]
    cliques.append(c)
print(cliques)
    

[[1, 13, 72, 36], [79, 19, 24, 37]]


In [5]:
with sqlite3.connect(str(db_file)) as con:
    cur = con.cursor()
    cur.execute('DELETE FROM matching')
    [cur.execute('INSERT INTO matching (article_id, group_id) VALUES (?, ?)', (a, i))
     for i, c in enumerate(cliques)
     for a in c]
    con.commit()

In [6]:
df

,article_id_1,article_id_2,permid,sklearn,source_id_1,source_id_2
1,31,3,0,0.628462,4,1
2,73,52,0,0.627555,4,2
3,11,4,0,0.626515,2,1
4,73,4,0,0.616885,4,1
8,25,17,0,0.592265,3,2
9,11,3,0,0.591422,2,1
10,32,3,0,0.590188,4,1
11,33,3,0,0.583866,4,1
12,32,4,0,0.581381,4,1
13,77,53,0,0.572215,4,2


In [7]:
cliques

[[1, 13, 72, 36], [79, 19, 24, 37]]